In [1]:
import numpy as np
from torch_geometric.data import Data
from torch_geometric.nn import radius_graph
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch.optim import Adam
import torch
import torch.nn.functional as F
from torch.nn import Linear, Module
from sklearn.preprocessing import StandardScaler

In [3]:
# Load the .npz file
data = np.load('C:/Users/nchni/OneDrive/Desktop/M1/Material Informatics/datasets/DFT_all.npz',allow_pickle=True)

In [5]:
Zs = data['atoms']             # atomic numbers (per molecule)
Rs = data['coordinates']       # 3D positions (per molecule)

# Extract dipole components
dipoles = data['dipole']
dipole_x = np.array([d[0] for d in dipoles])
dipole_y = np.array([d[1] for d in dipoles])
dipole_z = np.array([d[2] for d in dipoles])

# Define targets
targets = ['U0', 'gap', 'H']
Y_all = np.column_stack([
    data['U0'], data['gap'], data['H'],
    dipole_x, dipole_y, dipole_z
])

In [7]:
scaler = StandardScaler()
Y_scaled = scaler.fit_transform(Y_all)
Ys = list(Y_scaled)

In [9]:
def build_graph(Z, R, y):
    x = torch.tensor(Z, dtype=torch.float).reshape(-1, 1)
    pos = torch.tensor(R, dtype=torch.float)
    
    y = torch.tensor(y, dtype=torch.float).reshape(1, -1)  # ✅ fix: ensures [1, 6] shape
    
    edge_index = radius_graph(pos, r=5.0, loop=False)
    return Data(x=x, pos=pos, edge_index=edge_index, y=y)

In [11]:
dataset = [build_graph(z, r, y) for z, r, y in zip(Zs, Rs, Ys)]
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [13]:
class GNNModel(Module):
    def __init__(self, hidden_dim=64, output_dim=6):
        super().__init__()
        self.conv1 = GCNConv(1, hidden_dim)  # input: atomic number
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.lin1 = Linear(hidden_dim, hidden_dim)
        self.lin2 = Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)       # Global graph representation
        x = F.relu(self.lin1(x))
        out = self.lin2(x)                   # Multi-target output
        return out

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNNModel().to(device)
optimizer = Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.MSELoss()

In [ ]:
# Send model to train mode
model.train()

for epoch in range(1, 51):
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)

        # Forward pass
        pred = model(batch)  # shape [batch_size, 6]
        target = batch.y     # shape [batch_size, 6]

        # Confirm shapes match
        if pred.shape != target.shape:
            print(f"Shape mismatch: pred={pred.shape}, target={target.shape}")
            continue  # skip batch or raise an error

        # Loss and backprop
        loss = loss_fn(pred, target)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item() * batch.num_graphs

    avg_loss = total_loss / len(loader.dataset)
    print(f"Epoch {epoch:02d} | Loss: {avg_loss:.4f}")

Epoch 01 | Loss: 0.7660
Epoch 02 | Loss: 0.7295
Epoch 03 | Loss: 0.7201
Epoch 04 | Loss: 0.7161
Epoch 05 | Loss: 0.7137
Epoch 06 | Loss: 0.7124
Epoch 07 | Loss: 0.7110
Epoch 08 | Loss: 0.7098
Epoch 09 | Loss: 0.7088
Epoch 10 | Loss: 0.7080
Epoch 11 | Loss: 0.7075
Epoch 12 | Loss: 0.7074
Epoch 13 | Loss: 0.7068
Epoch 14 | Loss: 0.7066
Epoch 15 | Loss: 0.7063
Epoch 16 | Loss: 0.7063
Epoch 17 | Loss: 0.7057
Epoch 18 | Loss: 0.7053
Epoch 19 | Loss: 0.7052
Epoch 20 | Loss: 0.7051
Epoch 21 | Loss: 0.7049


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Step 6: Evaluate the model
model.eval()
all_preds, all_targets = [], []

with torch.no_grad():
    for batch in loader:
        batch = batch.to(device)
        pred = model(batch)
        all_preds.append(pred.cpu().numpy())
        all_targets.append(batch.y.view(pred.shape).cpu().numpy())

y_pred = np.vstack(all_preds)
y_true = np.vstack(all_targets)

# Inverse scale the predictions and targets
y_pred_real = scaler.inverse_transform(y_pred)
y_true_real = scaler.inverse_transform(y_true)

# Metrics
mae = mean_absolute_error(y_true_real, y_pred_real)
rmse = mean_squared_error(y_true_real, y_pred_real, squared=False)

print("\nFinal Evaluation:")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")